# Geo-Distribution of Tourism #

The tourism industry has been masively affected by the Covid-19 situation. One of the indicators for travelling is the number of Airbnb reviews, treated here as demand.

This notebook extracts the data from InsideAirbnb, aggregates it and displays the number of reviews for each neighbourhood per month.

### Inputs: ###
<br>Bristol_reviews.csv - This dataset is the reviews file downloaded from http://insideairbnb.com/get-the-data.html. It contains all the reviews up to the last scraped date. <br/>
<br> Bristol_listings.csv - This dataset is the listings file downloaded from http://insideairbnb.com/get-the-data.html. It contains the listings of that specific month. <br/>
<br>neighbourhoods.geojson - This dataset is the geojson file downloaded from http://insideairbnb.com/get-the-data.html. It contains the geometry of the neighbourhoods.


### Steps ###

1. Get the number of reviews per day per city.
2. Get the number of reviews per day per district.
3. Normalize teh number of reviews for each month.
4. Plot each neighbourhood for a given month.

In [ ]:
# Install all the dependencies

!pip install shapely
!pip install geopandas
!pip install geojsonio
!pip install descartes
!pip install wget
!pip install pandas

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import io
import wget
from os.path import isfile
from os import remove

In [ ]:
# Scrap the airbnb website and fetch the name and links to all available data files

r = requests.get("http://insideairbnb.com/get-the-data.html")
soup = BeautifulSoup(r.text)
alldata = {}

for table in tqdm(soup.findAll("table", {"class":"table-hover"})):
    df = pd.read_html(io.StringIO(str(table)))[0]
    df["downloadlink"] = ""
    index = 0
    for link in table.findAll("a",href=True):
        df.at[index,"downloadlink"] = link["href"]
        index += 1
    city = df["Country/City"].unique()[0]
    alldata[city] = df

## Get reviews per day per city

In [ ]:
def get_reviews_for_city(city_name):
    city_name = city_name.replace(" ", "_")
    filename = f"{city_name}_reviews.csv"
    if not isfile(filename):
        return None
    df = pd.read_csv(filename)    
    df["number_of_reviews"] = 1
    df = df[["date", "number_of_reviews"]]\
            .groupby("date")\
            .sum()\
            .reset_index(drop=False)\
            .rename(columns={
                "number_of_reviews": city_name}
            )
    df = df[df["date"] != "0"]
    df.date = pd.to_datetime(df.date)
    df.set_index("date", drop=True, inplace=True)
    df = df.astype(float)
    df = df.resample('D').mean()
    t_index = pd.DatetimeIndex(start='2014-01-01', end='2020-06-30', freq='D')
    df = df.interpolate(method='linear', limit_area="inside")
    return df

In [ ]:
# Get all the reviews for all the cities available on insideairbnb in one DataFrame

city_names = list(alldata.keys())
df_final = pd.DataFrame()
for idx, city_name in enumerate(city_names):
    print("City: ", city_name)
    if idx > 1000000:
        break
    df = get_reviews_for_city(city_name)
    if df is not None:
        df_final = pd.concat([df_final, df], axis=1)

In [ ]:
# Plot the number of reviews over time for Bristol and London

fig = df_final[["Bristol", "London"]].plot(figsize=(20, 10),fontsize= 16)
fig.legend(fontsize =16)

## Get reviews per day per district

In [ ]:
# Define utility functions

def get_geojson_url(city_name):
    df_city_info = alldata[city_name]
    df_city_info = df_city_info[df_city_info["File Name"] == "neighbourhoods.geojson"]
    return df_city_info["downloadlink"].iloc[0]

def download_geojson(geojson_url):
    if isfile("neighbourhoods.geojson"):
        remove("neighbourhoods.geojson")
    wget.download(geojson_url)

In [ ]:
# Get a dataframe with all the reviews of a given city, per district.

def get_reviews_by_district(city_name):
    
    geojson_url = get_geojson_url(city_name)
    download_geojson(geojson_url)
    if not isfile('neighbourhoods.geojson'):
        return None
    df_places = gpd.read_file('neighbourhoods.geojson')
    df_places["neighbourhood"] = df_places["neighbourhood"].astype(str)
    
    city_name = city_name.replace(" ", "_")
    filename = f"{city_name}_listings.csv"
    if not isfile(filename):
        return None
    df = pd.read_csv(filename)
    df = df[["id", "neighbourhood_cleansed"]]
    df.drop_duplicates(subset=["id"], keep="first", inplace=True)
    df["neighbourhood_cleansed"] = df["neighbourhood_cleansed"].astype(str)

    df = pd.merge(df, df_places, left_on="neighbourhood_cleansed", right_on="neighbourhood", how="left")
    df.fillna(0, inplace=True)
    df = df[["id", "neighbourhood", "geometry"]]
    
    filename = f"{city_name}_reviews.csv"
    if not isfile(filename):
        return None
    df_reviews = pd.read_csv(filename) 
    df = pd.merge(df, df_reviews, left_on="id", right_on="listing_id", how="right")
    df["yearmonth"] = df["date"].apply(lambda x: str(x)[:7])
    print(df)
    df.drop(["listing_id", "date"], axis=1, inplace=True)

    df["n_reviews"] = 1
    df = df.groupby(["neighbourhood", "yearmonth"]).agg({
            "geometry": lambda x: x.iloc[0],
            "n_reviews": sum
        }).reset_index(drop=False)

    return df


In [ ]:
df = get_reviews_by_district("Bristol")

## Normalize the number of reviews for each month##

In [ ]:
for yearmonth in df.yearmonth.unique():
    idx = df[df["yearmonth"] == yearmonth].index
    _sum = df[df["yearmonth"] == yearmonth]["n_reviews"].sum()
    df.iloc[idx, 3] = 100 * (df.iloc[idx, 3] / _sum)

## Plot each neighbourhood for a given month##

In [ ]:
# Choose a month and plot the map of the chosen city to see how many bookings were made in each district

MONTH = "2020-04"
gdf = gpd.GeoDataFrame(df[df["yearmonth"]==MONTH], 
                       geometry="geometry")
gdf.plot(column="n_reviews", 
         figsize=(20, 20),
         cmap="Blues", 
         linewidth=0.8, 
         edgecolor="0.8")
